In [1]:
# I want to extract the review score from GameSpot

import requests
from bs4 import BeautifulSoup

# Most websites refuse GET requests from python, so we change the header to pretend we're a browser.
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}
# get webpage
page = requests.get("https://ca.ign.com/search?q=Super%20Mario%20Kart&page=0&count=10&", headers = headers)
# page = requests.get("https://ca.ign.com/search?q=red%20dead%20redemption%202", headers = headers)
# print status. 200 is successful, without the headers we would get 403 forbidden 
page

<Response [200]>

In [2]:
# Show Content
page.content

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "https://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="https://www.w3.org/1999/xhtml" xml:lang="en" lang="en"\n      xmlns:og="https://ogp.me/ns#"\n      xmlns:fb="https://www.facebook.com/2008/fbml">\n<head>\n        <title>Search Results for &quot;Super Mario Kart&quot; - IGN</title>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<meta name="description" content="Product search results for &amp;quot;Super Mario Kart&amp;quot; on IGN" />\n<meta name="robots" content="noodp, noydir" />\n<meta name="copyright" content="IGN Entertainment, Inc." />\n<link rel="canonical" href="https://ca.ign.com/search" />\n<link rel="alternate" hreflang="en-au" href="https://au.ign.com/search" />\n<link rel="alternate" hreflang="en-ca" href="https://ca.ign.com/search" />\n<link rel="alternate" hreflang="en-ie" href="https://ie.ign.com/search" />\n<link rel="alternate" hreflang="en-gb" href="https:

In [3]:
# Use BeautifulSoup to parse the HTML and make it more readable
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "https://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="en" xml:lang="en" xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml" xmlns:og="https://ogp.me/ns#">
 <head>
  <title>
   Search Results for "Super Mario Kart" - IGN
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="Product search results for &amp;quot;Super Mario Kart&amp;quot; on IGN" name="description"/>
  <meta content="noodp, noydir" name="robots"/>
  <meta content="IGN Entertainment, Inc." name="copyright"/>
  <link href="https://ca.ign.com/search" rel="canonical"/>
  <link href="https://au.ign.com/search" hreflang="en-au" rel="alternate"/>
  <link href="https://ca.ign.com/search" hreflang="en-ca" rel="alternate"/>
  <link href="https://ie.ign.com/search" hreflang="en-ie" rel="alternate"/>
  <link href="https://uk.ign.com/search" hreflang="en-gb" rel="alternate"/>
  <

In [4]:
# create a list with the web content separated into childs
listOne = list(soup.children)

# print type of items in the list
[type(item) for item in listOne]

[bs4.element.Doctype,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Comment]

In [5]:
# Looking which item contains the Review, apparently it's the third
# print(listOne[2])

# check the third item and assign it to a variable
soup = listOne[2]
listTwo = list(soup.children)

# print type of items in the list
[type(item) for item in listTwo]

[bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString]

In [6]:
# Repeating... look for review tag, get item to a list, list children, repeat
soup = listTwo[3]
myList = list(soup.children)

[type(item) for item in myList]


[bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString]

In [7]:
# Realize this is boring and could be automated...
# Use fnmatch to get the index of the item containing 'review-score'
import fnmatch

# for each item in the list, convert the item to a string, search the term and print the index
for idx, item in enumerate(myList):
    tagData = str(myList[idx])
    if fnmatch.fnmatch(tagData, '*review-score*'):
        print(idx)
        
# this is better but it's still taking too long

6


In [8]:
# define function to get the index just like before and return it
def getIndex(myList, search):
    for idx, item in enumerate(myList):
        tagData = str(myList[idx])
        if fnmatch.fnmatch(tagData, search):
            return idx
        
# define search term
search = '*review-score*'

# repeat the previous processes until our soup have only one value
while len(myList) > 1:
    soup = myList[getIndex(myList, search)]
    myList = list(soup.children)

In [9]:
text = str(myList)
print(text)

text = text[4:28:]
text.strip()

['\n            9.5          ']


'9.5'